## Build an neural network in PyTorch

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [6]:
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]

In [9]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [14]:
class PimaClassifier(nn.Module): 
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x)) 
        return x
model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [15]:
loss_fn = nn.BCELoss() # binary cross-entropy 
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
n_epochs = 100
batch_size = 10
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
         Xbatch = X[i:i+batch_size]
         y_pred = model(Xbatch)
         ybatch = y[i:i+batch_size]
         loss = loss_fn(y_pred, ybatch)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 99, latest loss 0.2430160939693451


In [18]:
# evaluate the model on the training data sample
# compute accuracy (no_grad is optional)
with torch.no_grad(): 
    y_pred = model(X)
accuracy = (y_pred.round() == y).float().mean() 
print(f"Accuracy {accuracy}")

Accuracy 0.76171875


In [19]:
# make probability predictions with the model
predictions = model(X)
# round predictions
rounded = predictions.round()

In [22]:
# make class predictions with the model
predictions = (model(X) > 0.5).int() 
for i in range(5):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.599998474121094, 0.6269999742507935, 50.0] => 1 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.600000381469727, 0.35100001096725464, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.299999237060547, 0.671999990940094, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.100000381469727, 0.16699999570846558, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.099998474121094, 2.2880001068115234, 33.0] => 1 (expected 1)
